# Berth Allocation Problem

## Util functions

In [1]:
# import sys
# !{sys.executable} -m pip install pulp
import random
from pulp import *

ModuleNotFoundError: No module named 'pulp'

In [ ]:
pip install pulp

## 1. generate input

In [50]:
L = random.randint(500, 1000)
print("Total length =", L)
n = random.randint(5, 15)
print("Number of vessels =", n)

days = random.randint(4, 8)
maxOperationTime = 48
penaltyLimit = 4
print("Assume days for allocation =", days)
print("Assume max operation time =", maxOperationTime)
print("Assume penalty limit =", penaltyLimit)
time = days * 24

p = [0] * n
a = [0] * n
b = [0] * n
d = [0] * n
l = [0] * n
c1 = [0] * n
c2 = [0] * n

for i in range(n):
    p[i] = random.randint(0, L)
    a[i] = random.randint(0, time)
    b[i] = random.randint(1, maxOperationTime)
    d[i] = random.randint(a[i] + 1, time)
    l[i] = random.randint(1, L)
    c1[i] = random.randint(1, 4)
    c2[i] = random.randint(1, 4)

Total length = 619
Number of vessels = 15
Assume days for allocation = 6
Assume max operation time = 48
Assume penalty limit = 4


## 2. Create model

In [51]:
M = 99999999999 # assign a max number, not sure if float("inf") will trigger some potential bugs
m = LpProblem("Berth Allocation Problem", LpMinimize)
x = [LpVariable("x{}".format(i), 0, L, LpInteger) for i in range(n)]
y = [LpVariable("y{}".format(i), 1, maxOperationTime, LpInteger) for i in range(n)]
E = [(i, j) for i in range(n) for j in range(n) if i != j]
zx = LpVariable.dicts("zx", E, cat=LpBinary)
zy = LpVariable.dicts("zy", E, cat=LpBinary)

Y = LpVariable("Y", cat=LpInteger)
Y_pos = LpVariable("Y_pos", cat=LpInteger)

part1 = LpVariable("part1", cat=LpInteger)
part2 = LpVariable("part2", cat=LpInteger)
# Create objective
m += part1 + part2, "Objective"

X = [LpVariable("X{}".format(i), cat=LpInteger) for i in range(n)]
X_abs = [LpVariable("X_abs{}".format(i), cat=LpInteger) for i in range(n)]
for i in range(n):
    m += X[i] == (x[i] - p[i])
    m += X_abs[i] >= X[i]
    m += X_abs[i] >= -X[i]
m += part1 == sum(c1[i] * X_abs[i] for i in range(n))


Y = [LpVariable("Y{}".format(i), cat=LpInteger) for i in range(n)]
Y_pos = [LpVariable("Y_pos{}".format(i), cat=LpInteger) for i in range(n)]
for i in range(n):
    m += Y[i] == (y[i] + b[i] - d[i])
    m += Y_pos[i] >= Y[i]
    m += Y_pos[i] >= 0
m += part2 == sum(c2[i] * Y_pos[i] for i in range(n))

# Add constraints
for i in range(n):
    m += x[i] + l[i] <= L #, "x right most end constraint"

for i in range(n):
    for j in range(n):
        if i != j:
            m += x[i] + l[i] <= x[j] + M * (1 - zx[i,j])#, "left hand constraint"
            m += y[i] + b[i] <= y[j] + M * (1 - zy[i,j])#, "time no later constraint"
for i in range(n):
    for j in range(i):
        if i < j:
            m += zx[i,j] + zx[j,i] + zy[i,j] + zy[j,i] >= 1#, "no overlap constraint"
for i in range(n):
    m += y[i] >= a[i]#, "after arrive constraint"
for i in range(n):
    m += x[i] >= 0#, "non-negetive constraint"

## 3. solve

In [52]:
m.solve()

-1

## 4. present results

In [39]:
vobj = value(m.objective)